In [1]:
!pip install scenedetect
!pip3 install whisper-timestamped

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.0/800.0 kB 11.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240927-py3-none-any.whl size=802818 sha256=0dac3c876e2d3c52c83350eeaa6df56827f5e4abc68a92e48b1262737212231e
  Stored in directory: /root/.cache/pip/wheels/1c/d0/fd/81c5b31ba2016ac95f2f8e41d0e7016f2aab2a0bd306a7ab59
Successfully built openai-whi

In [8]:
from scenedetect import detect, AdaptiveDetector, split_video_ffmpeg
import numpy as np
import librosa
import whisper_timestamped as whisper
import torch

### Разделение на сцены

На выходе:
``` python
print(scene_list)

[(00:00:00.000 [frame=0, fps=30.000], 00:00:01.533 [frame=46, fps=30.000]),
 (00:00:01.533 [frame=46, fps=30.000], 00:00:02.133 [frame=64, fps=30.000]),
 (00:00:02.133 [frame=64, fps=30.000], 00:00:03.500 [frame=105, fps=30.000]),
 (00:00:03.500 [frame=105, fps=30.000], 00:00:04.267 [frame=128, fps=30.000])]
 ```

In [5]:
video_path = 'video1.mp4'
scene_list = detect(video_path, AdaptiveDetector())

INFO:pyscenedetect:Downscale factor set to 2, effective resolution: 320 x 180
INFO:pyscenedetect:Detecting scenes...


### Транскрибация

На входе:
- video_path
- scene_list



In [16]:
audio, sr = librosa.load(video_path)

audio_scenes = []
min_seconds = 2
for start_time, end_time in scene_list:

    start_frame = int(start_time.get_seconds() * sr)
    end_frame = int(end_time.get_seconds() * sr)

    if end_time.get_seconds()-start_time.get_seconds() > min_seconds:
      audio_scene = audio[start_frame:end_frame]
      audio_scenes.append(audio_scene)
    else:
      audio_scenes.append(np.array([]))

  audio, sr = librosa.load(video_path)

	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
model = whisper.load_model("openai/whisper-small", device=device)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



pytorch_model.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

  hf_state_dict = torch.load(model_path, map_location="cpu")



In [11]:
def ff(res):
    words = res['text'].split(' ')
    values, counts = np.unique(np.array(words), return_counts=True)
    return len(values)/len(words)

def f(res):
    try:
      secs = res['segments'][-1]['end'] - res['segments'][0]['start']
      words = len(res['text'].split(' '))
      return words / secs
    except:
      return 0

In [17]:
results = []
for i in range(len(audio_scenes)):
  if audio_scenes[i].shape[0] != 0:
    results.append(whisper.transcribe(model, audio_scenes[i]))
  else:
    results.append(None)

Detected language: Russian


100%|██████████| 289/289 [00:10<00:00, 27.13frames/s]


Detected language: Russian


100%|██████████| 519/519 [00:10<00:00, 51.77frames/s]


Detected language: Russian


...

Detected language: English


100%|██████████| 335/335 [00:09<00:00, 34.31frames/s]


Detected language: English


  0%|          | 0/298 [00:03<?, ?frames/s]


KeyboardInterrupt: 

In [18]:
# текст по сценам (размеченный)
results

[None,
 None,
 None,
 None,
 {'text': ' Сейчас находимся с Нилой Чайлдом.',
  'segments': [{'id': 0,
    'seek': 0,
    'start': 0.0,
    'end': 2.84,
    'text': ' Сейчас находимся с Нилой Чайлдом.',
    'tokens': [50364,
     23590,
     19363,
     32455,
     776,
     2410,
     2338,
     1700,
     7099,
     3183,
     693,
     856,
     1253,
     13,
     50506],
    'temperature': 0.0,
    'avg_logprob': -0.4675440788269043,
    'compression_ratio': 0.953125,
    'no_speech_prob': 0.03241311386227608,
    'confidence': 0.702,
    'words': [{'text': 'Сейчас',
      'start': 0.0,
      'end': 0.32,
      'confidence': 0.767},
     {'text': 'находимся', 'start': 0.32, 'end': 1.16, 'confidence': 0.971},
     {'text': 'с', 'start': 1.16, 'end': 1.78, 'confidence': 0.862},
     {'text': 'Нилой', 'start': 1.78, 'end': 2.16, 'confidence': 0.508},
     {'text': 'Чайлдом.', 'start': 2.16, 'end': 2.84, 'confidence': 0.708}]}],
  'language': 'ru',
  'language_probs': {'en': 0.008523877

In [19]:
# текст всего видео
video_text = ' '.join([i['text'] for i in results if i and ff(i)>0.2])
video_text

' Сейчас находимся с Нилой Чайлдом.  Кто первый двинется, тот гей. Инглиш или русский?  Какой язык предпочитаешь? Ирины Гришиллерумский  Maybe you got something that you know'